Problem :--->   Detection of Intent from  a given sentence

Data-loading and Preprocessing:-

     1.I Used filtering techniques like removing stop words filtering words using regular expressions
      But if I do like this I was drastically loosing my prediction accuracy its shown me that not only I am removing
      unimportant words but also loosing information.
      
     2.I Used Information Extraction Techniques like a Count Vectorizer,Tfidf,WordEmbeddings.Character lever information 
         Extraction Techniques.
         
     3.I used POS Tagging techique for grouping similar information.
     
     4.WordEmbeddings(By using word2vec and glove vectors I extracted Feature Matrix .npy file)
         For this I did another analysis in google collab for sentence embedding extraction.
         I used that Embedding in RNNs.
         Tried to build WordEmbeddings from FastText and ELMO MOdels sake of resource problem I didnot applied.
     5.Using analyzer I extracted character level and word level information this are available with 
             Vectorizer(tfidf,count)
             

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from keras.preprocessing import text, sequence
import nltk
nltk.download('averaged_perceptron_tagger')


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/harish/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
data = pd.read_csv('enron_train.csv',names = ['intent', 'Messages'],error_bad_lines=False)
test = pd.read_csv('enron_test.csv',names = ['intent', 'Messages'],error_bad_lines=False)
trainDF = pandas.DataFrame()
trainDF['text'] = data['Messages']
trainDF['label'] = data['intent']

In [3]:
testDF = pandas.DataFrame()
testDF['text'] = test['Messages']
testDF['label'] = test['intent']

In [4]:
# split the dataset into training and validation datasets 
##train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
train_x=trainDF['text']
valid_x=testDF['text']
train_y=trainDF['label']
valid_y=testDF['label']
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [5]:
trainDF['label'].value_counts()

No     1857
Yes    1719
Name: label, dtype: int64

In [6]:
testDF['label'].value_counts()

No     683
Yes    309
Name: label, dtype: int64

In [7]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [8]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

In [9]:
''' load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        '''

" load the pre-trained word-embedding vectors \nembeddings_index = {}\nfor i, line in enumerate(open('data/wiki-news-300d-1M.vec')):\n    values = line.split()\n    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')\n\n# create a tokenizer \ntoken = text.Tokenizer()\ntoken.fit_on_texts(trainDF['text'])\nword_index = token.word_index\n\n# convert text to sequence of tokens and pad them to ensure equal length vectors \ntrain_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)\nvalid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)\n\n# create token-embedding mapping\nembedding_matrix = numpy.zeros((len(word_index) + 1, 300))\nfor word, i in word_index.items():\n    embedding_vector = embeddings_index.get(word)\n    if embedding_vector is not None:\n        embedding_matrix[i] = embedding_vector\n        "

In [10]:
import numpy as np
##embedding_matrix.dump("/content/drive/My Drive/my_matrix.")
#numpy.save('/content/drive/My Drive/test', embedding_matrix)
##embedding_matrix= numpy.load("/content/drive/My Drive/my_matrix.dat")
embedding_matrix=np.load('test.npy')

In [11]:
embedding_matrix.shape

(5076, 300)

In [12]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [13]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

Machine learning Modeling Techiniques like a Discriminative  and Generative modeling techniques I used.

In [14]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [15]:
def train_model(classifier,feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    if is_neural_net==False:
        classifier.fit(feature_vector_train, label)
    else:
        classifier.fit(feature_vector_train, label,epochs=10)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y),predictions


In [16]:
# Naive Bayes on Count Vectors
accuracy,pred = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print "NB, Count Vectors: ", accuracy

# Naive Bayes on Word Level TF IDF Vectors
accuracy,pred = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print "NB, WordLevel TF-IDF: ", accuracy

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy,pred = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "NB, N-Gram Vectors: ", accuracy

# Naive Bayes on Character Level TF IDF Vectors
accuracy,pred = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print "NB, CharLevel Vectors: ", accuracy

NB, Count Vectors:  0.6975806451612904
NB, WordLevel TF-IDF:  0.6743951612903226
NB, N-Gram Vectors:  0.7348790322580645
NB, CharLevel Vectors:  0.717741935483871


In [17]:
# Linear Classifier on Count Vectors
accuracy,pred = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print "LR, Count Vectors: ", accuracy

# Linear Classifier on Word Level TF IDF Vectors
accuracy,pred = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print "LR, WordLevel TF-IDF: ", accuracy

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,pred = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "LR, N-Gram Vectors: ", accuracy

# Linear Classifier on Character Level TF IDF Vectors
accuracy,pred = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print "LR, CharLevel Vectors: ", accuracy

LR, Count Vectors:  0.7348790322580645
LR, WordLevel TF-IDF:  0.7832661290322581
LR, N-Gram Vectors:  0.7389112903225806
LR, CharLevel Vectors:  0.7449596774193549


In [18]:
# SVM on Ngram Level TF IDF Vectors
accuracy,pred = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "SVM, N-Gram Vectors: ", accuracy

SVM, N-Gram Vectors:  0.688508064516129


In [19]:
# RF on Count Vectors
accuracy,pred = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print "RF, Count Vectors: ", accuracy

# RF on Word Level TF IDF Vectors
accuracy,pred = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print "RF, WordLevel TF-IDF: ", accuracy

RF, Count Vectors:  0.7530241935483871
RF, WordLevel TF-IDF:  0.7479838709677419


In [20]:
# Extereme Gradient Boosting on Count Vectors
accuracy,pred = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print "Xgb, Count Vectors: ", accuracy

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy,pred = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print "Xgb, WordLevel TF-IDF: ", accuracy

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy,pred = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print "Xgb, CharLevel Vectors: ", accuracy



/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, Count Vectors:  0.7943548387096774


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, WordLevel TF-IDF:  0.8024193548387096
Xgb, CharLevel Vectors:  0.8084677419354839


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Deep learning Modeling Techniques:-
    

In [21]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy,pred = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print "NN, Ngram Level TF IDF Vectors",  accuracy

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
3576/3576 [==============================] - 1s - loss: 0.6698     
Epoch 2/10
3576/3576 [==============================] - 1s - loss: 0.5097     
Epoch 3/10
3576/3576 [==============================] - 1s - loss: 0.3549     
Epoch 4/10
3576/3576 [==============================] - 1s - loss: 0.2706     
Epoch 5/10
3576/3576 [==============================] - 1s - loss: 0.2203     
Epoch 6/10
3576/3576 [==============================] - 1s - loss: 0.1866     
Epoch 7/10
3576/3576 [==============================] - 1s - loss: 0.1640     
Epoch 8/10
3576/3576 [==============================] - 1s - loss: 0.1499     
Epoch 9/10
3576/3576 [==============================] - 1s - loss: 0.1381     
Epoch 10/10
3576/3576 [==============================] - 1s - loss: 0.1304     
NN, Ngram Level TF IDF Vectors 0.688508064516129


This is the Step for Converting Text to Sequence format for Deep learning  

In [22]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index
#token = text.Tokenizer()
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

In [23]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(5076, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy,pred = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "CNN, Word Embeddings",  accuracy

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
3576/3576 [==============================] - 5s - loss: 0.6206     
Epoch 2/10
3576/3576 [==============================] - 6s - loss: 0.5258     
Epoch 3/10
3576/3576 [==============================] - 6s - loss: 0.4665     
Epoch 4/10
3576/3576 [==============================] - 4s - loss: 0.4098     
Epoch 5/10
3576/3576 [==============================] - 5s - loss: 0.3528     
Epoch 6/10
3576/3576 [==============================] - 5s - loss: 0.2973     
Epoch 7/10
3576/3576 [==============================] - 5s - loss: 0.2476     
Epoch 8/10
3576/3576 [==============================] - 4s - loss: 0.2186     
Epoch 9/10
3576/3576 [==============================] - 6s - loss: 0.1763     
Epoch 10/10
3576/3576 [==============================] - 6s - loss: 0.1504     
CNN, Word Embeddings 0.688508064516129


In [24]:
def create_rnn_lstm():

    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(5076, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy,pred = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "RNN-LSTM, Word Embeddings",  accuracy

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/10
3576/3576 [==============================] - 17s - loss: 0.6445    
Epoch 2/10
3576/3576 [==============================] - 17s - loss: 0.5757    
Epoch 3/10
3576/3576 [==============================] - 16s - loss: 0.5483    
Epoch 4/10
3576/3576 [==============================] - 18s - loss: 0.5260    
Epoch 5/10
3576/3576 [==============================] - 18s - loss: 0.5111    
Epoch 6/10
3576/3576 [==============================] - 19s - loss: 0.4820    - ETA: 1s - loss
Epoch 7/10
3576/3576 [==============================] - 18s - loss: 0.4729    
Epoch 8/10
3576/3576 [==============================] - 16s - loss: 0.4532    
Epoch 9/10
3576/3576 [==============================] - 18s - loss: 0.4423    
Epoch 10/10
3576/3576 [==============================] - 18s - loss: 0.4328    
RNN-LSTM, Word Embeddings 0.688508064516129


In [25]:

def create_rnn_gru():

    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy,pred = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "RNN-GRU, Word Embeddings",  accuracy

Epoch 1/10
3576/3576 [==============================] - 14s - loss: 0.6623    
Epoch 2/10
3576/3576 [==============================] - 15s - loss: 0.5806    
Epoch 3/10
3576/3576 [==============================] - 13s - loss: 0.5509    
Epoch 4/10
3576/3576 [==============================] - 15s - loss: 0.5243    
Epoch 5/10
3576/3576 [==============================] - 15s - loss: 0.5073    
Epoch 6/10
3576/3576 [==============================] - 15s - loss: 0.4830    
Epoch 7/10
3576/3576 [==============================] - 15s - loss: 0.4645    
Epoch 8/10
3576/3576 [==============================] - 15s - loss: 0.4466    
Epoch 9/10
3576/3576 [==============================] - 15s - loss: 0.4233    
Epoch 10/10
3576/3576 [==============================] - 15s - loss: 0.3954    
RNN-GRU, Word Embeddings 0.688508064516129


In [26]:

def create_bidirectional_rnn():

    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy,pred = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "RNN-Bidirectional, Word Embeddings",  accuracy

Epoch 1/10
3576/3576 [==============================] - 28s - loss: 0.6659    
Epoch 2/10
3576/3576 [==============================] - 27s - loss: 0.5714    
Epoch 3/10
3576/3576 [==============================] - 28s - loss: 0.5446    
Epoch 4/10
3576/3576 [==============================] - 27s - loss: 0.5200    
Epoch 5/10
3576/3576 [==============================] - 28s - loss: 0.5040    
Epoch 6/10
3576/3576 [==============================] - 27s - loss: 0.4787    
Epoch 7/10
3576/3576 [==============================] - 28s - loss: 0.4639    
Epoch 8/10
3576/3576 [==============================] - 28s - loss: 0.4416    
Epoch 9/10
3576/3576 [==============================] - 29s - loss: 0.4179    
Epoch 10/10
3576/3576 [==============================] - 27s - loss: 0.4036    
RNN-Bidirectional, Word Embeddings 0.688508064516129


In [27]:

def create_rcnn():

    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy,pred = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "CNN, Word Embeddings",  accuracy

Epoch 1/10
3576/3576 [==============================] - 4s - loss: 0.6158     
Epoch 2/10
3576/3576 [==============================] - 5s - loss: 0.5266     
Epoch 3/10
3576/3576 [==============================] - 5s - loss: 0.4658     
Epoch 4/10
3576/3576 [==============================] - 6s - loss: 0.4064     
Epoch 5/10
3576/3576 [==============================] - 6s - loss: 0.3596     - ETA: 0s - loss: 0
Epoch 6/10
3576/3576 [==============================] - 5s - loss: 0.3017     
Epoch 7/10
3576/3576 [==============================] - 5s - loss: 0.2529     
Epoch 8/10
3576/3576 [==============================] - 6s - loss: 0.2095     
Epoch 9/10
3576/3576 [==============================] - 6s - loss: 0.1745     
Epoch 10/10
3576/3576 [==============================] - 5s - loss: 0.1572     
CNN, Word Embeddings 0.688508064516129


Final Results

From Above Analysis I came to the conclusion that XGboost with Charecter level Vectors Model Give good results 

In [28]:
# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy,pred = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print "Xgb, CharLevel Vectors: ", accuracy


Xgb, CharLevel Vectors:  0.8084677419354839


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
report = classification_report(valid_y, pred)
print(report)

             precision    recall  f1-score   support

          0       0.85      0.88      0.86       683
          1       0.71      0.65      0.68       309

avg / total       0.80      0.81      0.81       992



In [30]:
seed = 7
model = xgboost.XGBClassifier()
scoring = 'roc_auc'
results = cross_val_score(model,xtrain_tfidf_ngram_chars.tocsc(), train_y,cv=5, scoring=scoring)
print("AUROC:"+str(results.mean()))

AUROC:0.700464997731844


In [31]:
model.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
predicted = model.predict(xvalid_tfidf_ngram_chars.tocsc())
matrix = confusion_matrix(valid_y, predicted)
print(matrix)

[[602  81]
 [109 200]]


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


From The Above results we can say Model Can Able to Differenced messages containt intent with 70% Confidence.

F1 Score is Also Showing that How much Our model Confident to predict Based on Precision and Recall.
F1 -Score 81%

For Aditional Research Analysis You can see my other notebook(Research_roughwork.ipynb) or python file.